# Convert voc to yolo 

In [76]:
import os
import glob
from pathlib import Path
import xml.etree.ElementTree as ET
from os import listdir, getcwd
dirs = ['China', 'Czech', 'India', 'Japan', 'Norway', 'United_States']
classes = ['D00', 'D10', 'D20', 'D40', 'Block crack', 'Repair']
cwd = getcwd()

def getImagesInDir(dir_path):
    image_list = []
    directory = glob.glob(dir_path +'/train/images'+ '**/*.jpg')
    for filename in directory:
        image_list.append(filename.replace(os.sep, '/'))
    return image_list

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(dir_path, output_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]
    path_xml=dir_path + '/train/annotations/xmls/' + basename_no_ext + '.xml'
    in_file = open(path_xml)
    out_file = open(output_path + basename_no_ext + '.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    for obj in root.iter('object'):
        try:
            difficult = obj.find('difficult').text
            cls = obj.find('name').text
            if cls not in classes or int(difficult)==1:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))  
            bb = convert((w,h), b)
            out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
        except:
            continue
for dir_path in dirs:
    cwd = cwd.replace(os.sep, '/')
    full_dir_path = cwd + '/all_countries/' + dir_path
    output_path = full_dir_path +'/yolo/'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_paths = getImagesInDir(full_dir_path)
    list_file = open(full_dir_path +'_train' '.txt', 'w')
    for image_path in image_paths:
        list_file.write(image_path + '\n')
        convert_annotation(full_dir_path, output_path, image_path)
    list_file.close()

    print("Finished processing: " + dir_path)

Finished processing: China
Finished processing: Czech
Finished processing: India
Finished processing: Japan
Finished processing: Norway
Finished processing: United_States


In [ ]:
import os
import shutil
import random
import glob
from pathlib import Path
import xml.etree.ElementTree as ET
from os import listdir, getcwd
# dirs = ['China', 'Czech', 'India', 'Japan', 'Norway', 'United_States']\n",
dirs = ['China']
classes = ['D00', 'D10', 'D20', 'D40', 'Block crack', 'Repair']
cwd = getcwd()

def getImagesInDir(dir_path):
    directory = glob.glob(dir_path +'/train/images'+ '**/*.jpg')
    long = len(directory)
    print(int(long*0.1))
    aded_images = []
    for i in range(int(long*0.1)):
        random_file=random.choice(directory)
        aded_images.append(random_file.replace(os.sep, '/'))
        return aded_images
for dir_path in dirs:
    cwd = cwd.replace(os.sep, '/')
    full_dir_path = cwd + '/all_countries/' + dir_path
    output_path = cwd +'/new/'+dir_path+'/images/'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_paths = getImagesInDir(full_dir_path)
    for image_path in image_paths:
        shutil.copy(image_path, output_path)
    print("Finished processing:" + dir_path)